In [1]:
"""
    copy from https://qiita.com/wataoka/items/5c6766d3e1c674d61425
"""
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
import numpy as np
import os
import cv2
from PIL import Image

Using TensorFlow backend.
C:\euler\Anaconda3\envs\py36_django_mnist_git\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\euler\Anaconda3\envs\py36_django_mnist_git\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\euler\Anaconda3\envs\py36_django_mnist_git\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\euler\Anaconda

In [8]:
def conver_to_mnist(convas_jpg):
    # 画像が RGB ならグレースケールに変換する。(28, 28, 3) -> (28, 28)
    img_gray = convas_jpg.convert("L")
    size = (28, 28)
    img_gray = img_gray.resize(size)

    # 行列を1次元に変換する(28, 28) -> (28, 28, 1) にする。
    data = np.array(img_gray).reshape(28, 28, 1)
    mnist_data = 255 - data

    # img_mnist = Image.fromarray(mnist_data.reshape(28, -1))
    # img_mnist.save(os.path.join(img_save_path, '{}_input_minist_gray_2.png'.format(rdint)))

    mnist_data = mnist_data.astype('float32') / 255

    return mnist_data

# https://codeday.me/bug/20190505/1055129.html
def load_data(img_dir):
    # return np.array([cv2.imread(os.path.join(img_dir, img)).flatten() for img in os.listdir(img_dir) if img.endswith(".jpg")])

    list_of_imgs = []
    list_of_results = []
    for file_name in os.listdir(img_dir):
        full_name = os.path.join(img_dir, file_name)
        if not full_name.endswith(".jpg"):
            continue
        img = Image.open(full_name)
        if img is None:
            print("Unable to read image", full_name)
            continue
        
        mnist_data = conver_to_mnist(img)
        list_of_imgs.append(mnist_data)
        list_of_results.append(file_name[0])
        
    x = np.array(list_of_imgs)
    y =  np.array(list_of_results)

    return x, y

In [15]:
train_image="train_image"
(x_train, y_train) = load_data(train_image)
x_train = x_train.reshape(x_train.shape[0], -1)
# 正解ラベルをone-hot-encoding
y_train = to_categorical(y_train, 10)

print(x_train.shape)
print(y_train.shape)

test_image="test_image"
(x_test, y_test) = load_data(test_image)
x_test = x_test.reshape(x_test.shape[0], -1)
# 正解ラベルをone-hot-encoding
y_test = to_categorical(y_test, 10)

print(x_test.shape)
print(y_test.shape)

(1339, 784)
(1339, 10)
(343, 784)
(343, 10)


In [16]:
# 2.モデルを構築
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=784))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                330       
Total params: 111,146
Trainable params: 111,146
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
# 3.モデルにデータを学習させる
model.fit(x_train, y_train,
          batch_size=100,
          epochs=70,
          verbose=1)


Epoch 1/70
1339/1339 [==============================] - 0s 154us/step - loss: 2.2084 - acc: 0.2353
Epoch 2/70
1339/1339 [==============================] - 0s 24us/step - loss: 1.9511 - acc: 0.4324
Epoch 3/70
1339/1339 [==============================] - 0s 25us/step - loss: 1.7649 - acc: 0.4996
Epoch 4/70
1339/1339 [==============================] - 0s 28us/step - loss: 1.6156 - acc: 0.5541
Epoch 5/70
1339/1339 [==============================] - 0s 27us/step - loss: 1.4837 - acc: 0.5840
Epoch 6/70
1339/1339 [==============================] - 0s 26us/step - loss: 1.3710 - acc: 0.6318
Epoch 7/70
1339/1339 [==============================] - 0s 27us/step - loss: 1.2569 - acc: 0.6587
Epoch 8/70
1339/1339 [==============================] - 0s 27us/step - loss: 1.1586 - acc: 0.6938
Epoch 9/70
1339/1339 [==============================] - 0s 27us/step - loss: 1.0664 - acc: 0.7252
Epoch 10/70
1339/1339 [==============================] - 0s 28us/step - loss: 0.9926 - acc: 0.7341
Epoch 11/70
1339/1

In [18]:
# 4.モデルを評価する
score = model.evaluate(x_test, y_test)
print(score)

343/343 [==============================] - 0s 175us/step
[1.7635526497231975, 0.7201166175544783]


In [19]:
model.save("eulerlcs_ann.h5")